There we import tensorflow pre-trained models. Inception v4 and v2 are also available if needed

In [53]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import tensorflow.contrib.slim.nets as nets

import tempfile
from urllib.request import urlretrieve
import tarfile
import os

There we establish session for the whole notebook. On what is session and why tf.* code is not executed when you expect it to be read this article:

https://www.easy-tensorflow.com/tf-tutorials/basics/graph-and-session

In [2]:
tf.logging.set_verbosity(tf.logging.ERROR)
sess = tf.InteractiveSession()

There we declare `f.Variable`for our adversarial image. This type is used for all dynamically changing obects (such as weights of the model). Note that `f.Variable`actually does not store the value inside it, rather it stores *operations* allowing to get this value. To actually *init* `f.Variable` run `tf.global_varibles_initializer`. Remember that the size of the `tf.Variable` cannot be changed dinamically in the code. One can change its dimensionality, though. More about `tf.Variables` and the difference between them, `tf.constant` and `tf.placeholder` can be found here:

https://www.easy-tensorflow.com/tf-tutorials/basics/graph-and-session

In [52]:
image = tf.Variable(tf.zeros((299, 299, 3)))

# code to print image as an array
# sess.run(tf.global_variables_initializer())
# print(image.eval())

tf.expand_dims(tf.Variable, axis=?) is equivalent to np.reshape(np.ndarray), but only adds one dimesion and numeration of axis is from the left to the right. Arg_scope downloads default parameters of the model (such as size of convolutional regions). 

In [ ]:
def inception(image, reuse):
    preprocessed = tf.multiply(tf.subtract(tf.expand_dims(image, 0), 0.5), 2.0)
    arg_scope = nets.inception.inception_v3_arg_scope(weight_decay=0.0)
    with slim.arg_scope(arg_scope):
        logits, _ = nets.inception.inception_v3(
            preprocessed, 1001, is_training=False, reuse=reuse)
        logits = logits[:,1:] # ignore background class
        probs = tf.nn.softmax(logits) # probabilities
    return logits, probs

logits, probs = inception(image, reuse=False)

Download pretrained weights

In [54]:
data_dir = tempfile.mkdtemp()
inception_tarball, _ = urlretrieve(
    'http://download.tensorflow.org/models/inception_v3_2016_08_28.tar.gz')
tarfile.open(inception_tarball, 'r:gz').extractall(data_dir)